# Importing the necessary libraries

In [16]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime,date
from dash.dependencies import Output, Input

# Importing the predictions dataset

In [17]:
today_date = date.today().strftime("%Y-%m-%d")
today_prediction = pd.read_csv(f"D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\stock_prices_predictions\\Predictions_{today_date}.csv")

In [19]:
past_prices = pd.read_csv(f"D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\stock_prices_dataset\\candle_stick\\Stocks_data_{today_date}.csv")
past_prices = past_prices[past_prices['Date']<today_date]

In [21]:
past_prices

,Date,Stock,Open,High,Low,Close
1,2025-06-04,ASHOKLEY,237.90,238.10,234.90,235.50
2,2025-06-03,ASHOKLEY,237.75,237.75,235.80,236.05
3,2025-06-02,ASHOKLEY,237.15,237.60,234.40,236.30
4,2025-05-30,ASHOKLEY,240.05,240.75,235.15,236.00
5,2025-05-29,ASHOKLEY,238.65,241.30,238.10,240.65
...,...,...,...,...,...,...
3682,2022-06-10,SBIN,465.00,465.00,458.80,461.80
3683,2022-06-09,SBIN,469.70,471.15,462.65,466.85
3684,2022-06-08,SBIN,467.40,475.50,463.25,471.05
3685,2022-06-07,SBIN,462.00,465.95,461.05,463.35


# Creating the dashboard

In [ ]:
app = Dash(__name__,external_stylesheets=[dbc.themes.COSMO])
server = app.server
stocks = ['ASHOKLEY','CANBK','LICI','ONGC','SBIN']
option_list = []

for stock in stocks:
    params = {'label':stock,'value':stock}
    params.append(option_list)
    
dropdown = html.Div([
    html.Label('Select the stock in BSE which you want to analyze',style={'display':'block','fontSize':15,'marginLeft':10}),
    dcc.Dropdown(
        id = 'stock',
        option = option_list,
        value = 'ASHOKLEY',
        clearable = False,
        style = {'marginLeft' : 10, 'marginRight' : 20}
    )
button =  html.Div([
    html.Button('Recalibrate model',id='recalibrate',style={'marginLeft': 10, 'marginRight': 20})
])

app.layout = html.Div(
    [
        dbc.Row([
            dbc.Col((html.H1('Stock market predictor',
                            style={'textAlign':'center','color':'white','marginTop':90})), width=12
                    ],style={'background-color':'indigo','marginBottom':20,'height':200}),
        html.Div(
        [
            dbc.Row([
                dbc.Col(dropdown, width=12)
            ],style={'marginBottom':30}),
            dbc.Row([
                dbc.Col(dcc.Graph(id='candlestick-chart',figure={},config={'displayModeBar':False})),
                dbc.Col(html.Div([
                    dbc.Row([
                        dbc.Col(html.H3('Expected price today'))
                    ]),
                    dbc.Row([
                        dbc.Col(html.H5(id='predicted-price',children=float(today_prediction.loc[:,'ASHOKLEY'].values[0])))
                    ])
            ], width=4,style={'textAlign': 'Right'}))
            ]),
            dbc.Row([
                dbc.Col(button)
            ])
        ])
    ]
)

@app.callback(
    Output(component_id='candlestick-chart', component_property='figure'),
    [Input(component_id='stock', component_property='value')]
)
def candlestick_chart(stock):
    past_prices_stock = past_prices[past_prices['Stock']==stock]
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=pd.to_datetime(past_prices_stock['Date']),open=past_prices_stock['Open'],high=past_prices_stock['High'],
                             low=past_prices_stock['Low'],close=past_prices_stock['Close'],increasing=dict(line=dict(color='blue')),
                             decreasing=dict(line=dict(color='red')),name='Past_prices'))
    return fig


    
        
            